### 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거한다.
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동평균선, 상단밴드, 하단밴드 파생변수 생성
2. 두 번째 함수를 생성(매개변수, 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은 ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print 사용)
3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익률 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익을 변경
    3. 누적 수익률을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익률을 출력(print 사용)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# 1번 함수 생성
# 매개 변수 추가
def create_band(df, col, start, end):
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)
    # start, end 를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    # 데이터를 시작시간부터 종료 시간까지 필터
    df = df.loc[start : end]
    # 결측치와 이상치를 제거
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis= 'columns'),[col]]
    # 수정 종가 컬럼을 제외한 데이터프레임 생성
    # df = df[[col]]
    # 이동평균선 생성
    df['center'] = df.rolling(20).mean() # ['Adj Close'] 생략
    # 상단 밴드 생성
    df['ub'] = df['center'] + (2 * df[col].rolling(20).std())
    # 하단 밴드 생성
    df['lb'] = df['center'] - (2 * df[col].rolling(20).std())
    # 결과를 리턴
    return df

In [3]:
load_data = pd.read_csv("../csv/GM.csv", index_col = 'Date')

In [6]:
data = create_band(load_data, 'Close', '20110101', '20150101')

In [7]:
data.iloc[18:25]

,Close,center,ub,lb
Date,,,,
2011-01-28,36.599998,NaN,NaN,NaN
2011-01-31,36.490002,37.9425,39.434780,36.450220
2011-02-01,36.450001,37.9120,39.501963,36.322037
2011-02-02,35.680000,37.8010,39.678466,35.923534
2011-02-03,36.060001,37.7005,39.726639,35.674361
2011-02-04,36.590000,37.5850,39.586447,35.583554
2011-02-07,36.700001,37.4710,39.396167,35.545833


In [9]:
data2 = create_band(load_data, 'Adj Close')

TypeError: create_band() missing 2 required positional arguments: 'start' and 'end'

In [8]:
create_band(data, col).iloc[18:25]

NameError: name 'col' is not defined

In [10]:
# 2번 함수 생성
def add_trade(df):
    # 기준이 되는 컬럼이 무엇인가
    # 기준이 되는 컬럼은 컬럼 중에 첫번째 이기 때문에 df.columns[0]
    col = df.columns[0]
    # trade 파생변수 생성
    df['trade']=""
    for i in df.index:
        # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, col] > df.loc[i, 'ub']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 매도
                df.loc[i, 'trade'] = ''
            else:
                df.loc[i, 'trade'] = ''
        # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i, col] < df.loc[i, 'lb']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태 유지
                df.loc[i, 'trade'] = 'buy'
            # 구매 상태가 아니면
            else:
                # 매수
                df.loc[i, 'trade'] = 'buy'
        else:
            # 현재 구매 상태면 
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            # 구매 상태가 아니면
            else:
                df.loc[i, 'trade'] = ''
    return df

In [ ]:
add_trade(data)['trade'].value_counts()

trade
buy    1187
        975
Name: count, dtype: int64

In [11]:
add_trade(data2)['trade'].value_counts()

NameError: name 'data2' is not defined

In [ ]:
data =  add_trade(data)

In [12]:
# 3번째 함수 생성
def add_rtn(df):
    col = df.columns[0]
    # 수익율 파생변수 생성
    df['return'] = 1
    # 판매한 날의 수익율 대입
    rtn = 1.0
    buy = 0.0
    sell= 0.0

    for i in df.index:
        # 구매가를 출력
        if (df.shift(1).loc[i, 'trade'] == '') and (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, col]
            print('진입일 :', i, '구매가격 :', buy)
        # 판매가를 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, col]
            rtn = (sell - buy) / buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :',  i, '판매가격 :', sell, '수익율 :', rtn)

        # 구매가, 판매가를 초기화
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
    # 누적 수익율 계산하여 파생변수에 대입
    acc_rtn = 1.0

    for i in df.index:
        rtn = df.loc[i,'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn

    print('누적 수익율 :', acc_rtn)

    return df

In [13]:
add_rtn(data)

KeyError: 'trade'

In [88]:
import bollinger as bg
import imp
imp.reload(bg)

<module 'bollinger' from '/Users/noseohyeong/Desktop/유비온/파이썬 실습/20230413/bollinger.py'>

In [ ]:
df = pd.read_csv("../csv/MSFT.csv", index_col='Date')

In [ ]:
# 1번 함수를 실행
data = bg.create_band(df, 'Adj Close', '20100101', '20190101')

In [ ]:
# 2번 함수를 실행
data2 = bg.add_trade(data)

In [ ]:
# 3번 함수를 실행
bg.add_rtn(data2)

진입일 : 2010-01-22 00:00:00 구매가격 : 22.948128
판매일 : 2010-03-17 00:00:00 판매가격 : 23.588829 수익율 : 1.0279195322598862
진입일 : 2010-05-06 00:00:00 구매가격 : 23.07136
판매일 : 2010-09-13 00:00:00 판매가격 : 20.187721 수익율 : 0.875012179602763
진입일 : 2010-11-23 00:00:00 구매가격 : 20.319853
판매일 : 2011-01-06 00:00:00 판매가격 : 23.312819 수익율 : 1.1472926994107686
진입일 : 2011-03-16 00:00:00 구매가격 : 20.171436
판매일 : 2011-04-06 00:00:00 판매가격 : 21.278055 수익율 : 1.0548606950937949
진입일 : 2011-05-16 00:00:00 구매가격 : 19.992422
판매일 : 2011-06-27 00:00:00 판매가격 : 20.639456 수익율 : 1.032363962705469
진입일 : 2011-08-04 00:00:00 구매가격 : 21.245531
판매일 : 2011-08-31 00:00:00 판매가격 : 21.923595 수익율 : 1.0319156061573607
진입일 : 2011-11-01 00:00:00 구매가격 : 21.420837
판매일 : 2012-01-03 00:00:00 판매가격 : 22.229851 수익율 : 1.037767618510892
진입일 : 2012-04-04 00:00:00 구매가격 : 26.087456
판매일 : 2012-06-19 00:00:00 판매가격 : 25.829544 수익율 : 0.9901135626256542
진입일 : 2012-07-12 00:00:00 구매가격 : 24.087934
판매일 : 2012-09-06 00:00:00 판매가격 : 26.551155 수익율 : 1.1022595379080664
진입일 :

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,24.525019,24.056307,24.964478,23.148135,,1.0,1.000000
2010-01-05,24.532942,24.095134,25.015633,23.174635,,1.0,1.000000
2010-01-06,24.382378,24.133962,25.032815,23.235109,,1.0,1.000000
2010-01-07,24.128809,24.168828,25.004852,23.332803,,1.0,1.000000
2010-01-08,24.295214,24.206467,24.989876,23.423058,,1.0,1.000000
...,...,...,...,...,...,...,...
2018-12-24,93.384033,105.437251,114.470702,96.403801,buy,1.0,2.619028
2018-12-26,99.763077,105.144093,114.525564,95.762621,buy,1.0,2.619028
2018-12-27,100.378166,104.848455,114.447882,95.249027,buy,1.0,2.619028


In [96]:
data = pd.read_csv("../csv/AAPL.csv", index_col='Date')

In [97]:
a = bg.Invest(data, 'Adj Close')

In [98]:
df = a.create_band('20100101', '20150101')

In [99]:
a.df

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [100]:
a.add_trade

<bound method Invest.add_trade of <bollinger.Invest object at 0x11da9e3a0>>

In [101]:
# 클래스 생성
class_a = bg.Invest(data, 'Adj Close')

In [103]:
class_a.col

'Adj Close'

In [104]:
class_a.df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.360548,26432000.0
1980-12-17,0.462054,0.464286,0.462054,0.462054,0.369472,21610400.0
1980-12-18,0.475446,0.477679,0.475446,0.475446,0.380182,18362400.0
...,...,...,...,...,...,...
2019-06-18,196.050003,200.289993,195.210007,198.449997,198.449997,26551000.0
2019-06-19,199.679993,199.880005,197.309998,197.869995,197.869995,21124200.0
2019-06-20,200.369995,200.610001,198.029999,199.460007,199.460007,21514000.0


In [105]:
class_a.create_band('20100101', '20150101')
# 한번더 실행하면 안됨 이유는?

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [106]:
class_a.add_trade()

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [107]:
class_a.df

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [108]:
class_a.add_rtn()

진입일 : 2010-01-22 00:00:00 구매가격 : 24.747818
판매일 : 2010-03-01 00:00:00 판매가격 : 26.154476 수익율 : 1.0568396777445188
진입일 : 2010-08-24 00:00:00 구매가격 : 30.026524
판매일 : 2010-09-08 00:00:00 판매가격 : 32.90366 수익율 : 1.0958198158401553
진입일 : 2011-03-16 00:00:00 구매가격 : 41.299767
판매일 : 2011-07-01 00:00:00 판매가격 : 42.957966 수익율 : 1.040150323366231
진입일 : 2011-11-14 00:00:00 구매가격 : 47.463268
판매일 : 2011-12-27 00:00:00 판매가격 : 50.876015 수익율 : 1.0719029081604747
진입일 : 2012-04-16 00:00:00 구매가격 : 72.601524
판매일 : 2012-06-18 00:00:00 판매가격 : 73.308609 수익율 : 1.0097392583659814
진입일 : 2012-10-08 00:00:00 구매가격 : 80.207954
판매일 : 2013-07-29 00:00:00 판매가격 : 57.243137 수익율 : 0.7136840443530077
진입일 : 2013-09-11 00:00:00 구매가격 : 60.184383
판매일 : 2013-10-18 00:00:00 판매가격 : 65.48336 수익율 : 1.0880457144505413
진입일 : 2014-01-03 00:00:00 구매가격 : 70.019096
판매일 : 2014-03-25 00:00:00 판매가격 : 70.960335 수익율 : 1.0134426042861222
진입일 : 2014-10-15 00:00:00 구매가격 : 89.842468
판매일 : 2014-10-23 00:00:00 판매가격 : 96.557182 수익율 : 1.0747387527243797
진입일 

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717


In [109]:
class_a.df

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717
